# El banco portugués está teniendo una disminución en sus ingresos, por lo que quieren poder identificar a los clientes existentes que tienen una mayor probabilidad de contratar un depósito a largo plazo. Esto permitirá que el banco centre sus esfuerzos de marketing en esos clientes y evitará perder dinero y tiempo en clientes que probablemente no se suscribirán.

Para abordar este problema crearemos un algoritmo de clasificación que ayude a predecir si un cliente contrará o no un depósito a largo plazo.

* Importación de librerias.
* Lectura de datos.
* Limpieza de datos.
* Verificación de variables.


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error , mean_absolute_percentage_error , accuracy_score, classification_report
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from pickle import dump

In [2]:
data_original = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv", sep=";")
data_original.tail()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41187,74,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,3,999,1,failure,-1.1,94.767,-50.8,1.028,4963.6,no


In [3]:
data_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [4]:
data_original.nunique()

age                 78
job                 12
marital              4
education            8
default              3
housing              3
loan                 3
contact              2
month               10
day_of_week          5
duration          1544
campaign            42
pdays               27
previous             8
poutcome             3
emp.var.rate        10
cons.price.idx      26
cons.conf.idx       26
euribor3m          316
nr.employed         11
y                    2
dtype: int64

In [5]:
data_original.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

In [6]:
data_original.shape

(41188, 21)

In [7]:
data_original.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [8]:
data_original.duplicated().sum()

12

In [9]:
data_original = data_original.drop_duplicates()

In [10]:
data_original.duplicated().sum()

0

Convertimos las variables para poder manejarlas y trabajar sobre sus datos.

In [11]:
dumt = data_original.select_dtypes(include=('object', 'category'))
df_m = data_original.select_dtypes(exclude=('object', 'category'))
dummy = pd.get_dummies(dumt, drop_first=True)
dummy = dummy.astype(int)  # Convertir las variables dummy a enteros (0 o 1)
data = pd.concat([df_m, dummy], axis=1)
data.head(1)


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success,y_yes
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,1,0,0


In [12]:
# Calcula la matriz de correlación
correlacion = data.corr()

# Selecciona las columnas con correlación superior al 30% con 'y_yes' (excluyendo la propia columna 'y_yes')
relevantes = correlacion.index[abs(correlacion['y_yes']) > 0.2]

# Agrega la columna 'y_yes' a las columnas relevantes si no está ya incluida
relevantes = relevantes.union(['y_yes'])

# Filtra el DataFrame original para incluir solo las columnas relevantes
data_fil = data[relevantes]

#Guardamos los datos limpios y precesados.
data_fil.to_csv("/workspaces/primer_ml/data/processed/data_limpia.csv")

In [13]:
df_corr = data_fil.corr()
df_corr.style.background_gradient(cmap='coolwarm').format(precision=2)

,duration,emp.var.rate,euribor3m,nr.employed,pdays,poutcome_success,previous,y_yes
duration,1.00,-0.03,-0.03,-0.04,-0.05,0.04,0.02,0.41
emp.var.rate,-0.03,1.00,0.97,0.91,0.27,-0.26,-0.42,-0.30
euribor3m,-0.03,0.97,1.00,0.95,0.30,-0.28,-0.45,-0.31
nr.employed,-0.04,0.91,0.95,1.00,0.37,-0.35,-0.50,-0.35
pdays,-0.05,0.27,0.30,0.37,1.00,-0.95,-0.59,-0.32
poutcome_success,0.04,-0.26,-0.28,-0.35,-0.95,1.00,0.52,0.32
previous,0.02,-0.42,-0.45,-0.50,-0.59,0.52,1.00,0.23
y_yes,0.41,-0.30,-0.31,-0.35,-0.32,0.32,0.23,1.00


Podemos validar cuales son las variables con mayor nivel de correlación, que seran en este caso con las que trabajaremos para el modelo.

Separamos los daos y escalamos los valores para que el modelo tenga la mayor precisión posible.

In [14]:
X = data_fil.drop(['y_yes'], axis=1)
y = data_fil['y_yes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
scaler = MinMaxScaler()

X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [16]:
modelo = LogisticRegression()
modelo.fit(X_train_sc, y_train)

LogisticRegression()

In [17]:
y_pred = modelo.predict(X_test_sc)
y_pred

array([1, 0, 0, ..., 0, 0, 0])

In [18]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      7265
           1       0.65      0.35      0.45       971

    accuracy                           0.90      8236
   macro avg       0.79      0.66      0.70      8236
weighted avg       0.89      0.90      0.89      8236



In [21]:
param_dist = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 10],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 300, 400, 500]
}

grid_search = RandomizedSearchCV(modelo, param_dist, cv=5, scoring='recall', n_iter=10, random_state=42)

grid_search.fit(X_train_sc, y_train)

best_params = grid_search.best_params_

print("Mejores hiperparámetros:", best_params)

y_pred = grid_search.predict(X_test_sc)

modelo_final = LogisticRegression(**best_params)

print(classification_report(y_test, y_pred))

Mejores hiperparámetros: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 400, 'C': 1}
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      7265
           1       0.66      0.37      0.48       971

    accuracy                           0.90      8236
   macro avg       0.79      0.67      0.71      8236
weighted avg       0.89      0.90      0.89      8236



In [20]:
dump(modelo_final, open("/workspaces/primer_ml/models/modelo_final.sav", "wb"))

### Conclusión 

Precision (Precisión):

* Precision para la clase 0 (clientes que probablemente no contratarán un depósito a largo plazo) es alta (0.92). Esto significa que cuando el modelo predice que un cliente no contratará un depósito a largo plazo, es probable que esté en lo correcto el 92% de las veces.

* Precision para la clase 1 (clientes que probablemente contratarán un depósito a largo plazo) es más baja (0.66). Esto indica que cuando el modelo predice que un cliente contratará un depósito a largo plazo, solo está en lo correcto el 66% de las veces.

Recall (Recuperación o Sensibilidad):

* Recall para la clase 0 es alto (0.97). Esto significa que el modelo identifica correctamente al 97% de los clientes que no contratarán un depósito a largo plazo.

F1-Score:

* F1-Score combina precision y recall en una sola métrica. Para la clase 0, el F1-Score es alto (0.95), lo que indica un buen equilibrio entre precision y recall. Sin embargo, para la clase 1.

* F1-Score es más bajo (0.48), sugiriendo que hay margen de mejora en la capacidad del modelo para identificar clientes que contratarán un depósito a largo plazo.
Accuracy (Exactitud):

La exactitud global del modelo es del 90%, lo cual es un buen indicador general de su rendimiento en todo el conjunto de datos.
Macro AVG y Weighted AVG:

Las métricas macro avg y weighted avg brindan promedios ponderados y no ponderados, respectivamente, de las métricas de las dos clases. Estos valores proporcionan una comprensión general del rendimiento del modelo en todo el conjunto de datos.